In [10]:
import os
import sys

In [11]:
IN_COLAB='google.colab' in sys.modules

In [17]:
if IN_COLAB:
    from google.colab import drive

    WORKING_FOLDER="/content/drive/MyDrive/unicamp/ia368v_dd/trabalho_final_UNICAMP-IR/artigo/unicamp_IR"

    drive.mount('/content/drive', force_remount=True)

    os.chdir(WORKING_FOLDER)

    TREC_EVAL="/content/drive/MyDrive/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval"
else:
    WORKING_FOLDER="/mnt/0060f889-4c27-409b-b0de-47f5427515e3/unicamp/ia368v_dd/unicamp_IR/"
    TREC_EVAL="/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval"

Mounted at /content/drive


In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500)

In [3]:
ALL_EVALUATIONS="tests/evaluation_15_retrievers_000_009_sampled_50_queries_gpt-4-1106-preview_20240125.tsv"

BM25_RUN="tests/200-queries-passagens-by-BM25-pt_v2_with_query_id_run.txt"
BM25_E5_RUN="tests/bm25_e5_docs_000_999_50_sampled_queries_run.txt"
BM25_MT5_RUN="tests/200-queries-passagens-by-BM25-mT5_v4_with_query_id_run.txt"
SPLADE_RUN="tests/splade_pt_result_with_clueweb_id_run.txt"
COLBERTX_RUN="tests/colbertx_all_znormalized_validation_scores_000_199_run.txt"

E5_RUN="tests/e5_docs_000_999_50_sampled_queries_run.txt"
E5_BASE_RUN="tests/e5_base_docs_000_999_50_sampled_queries_run.txt"
SPLADE_MT5_RUN="tests/splade_mt5_docs_000_099_50_sampled_queries_run.txt"

BM25_1M_RUN="tests/bm25_1M_docs_000_999_sampled_50_queries_run.txt"
BM25_1M_MASSIVEWEB_RUN="tests/bm25_massiveweb_1M_docs_000_999_sampled_50_queries_run.txt"

COLBERTX_1M_MASSIVEWEB_RUN="tests/colbertx_all_retrieval_1M_dataset_20240105_run.txt"

E5_COLBERTX_RRF_RUN="tests/colbertx_e5_RRF_000_999_sampled_50_queries_run.txt"
E5_SPLADE_RRF_RUN="tests/splade_e5_fixed_RRF_000_999_sampled_50_queries_run.txt"

In [4]:
ADAV2="tests/adav2_1M_docs_000_999_sampled_50_queries.tsv"

In [88]:
DATASET_1M_FIELDS=['bm25_1M_massiveweb', 'colbertx_1M_massiveweb', 'adav2']

In [6]:
def prepare_run_format(results_filename, add_index=True, score_field='score', invert_score=False, retriever_name="retriever"):

    results_df = pd.read_csv(results_filename, sep='\t')

    display(results_df.head())

    results_grouped_df = results_df.groupby("query_id")['passage_id'].count()

    how_many_passages_per_question = results_grouped_df.iloc[0]

    if invert_score:
        print("Inverting score values in '{}' field...".format(score_field))

        results_df['score_inv'] = 1 - results_df[score_field]
        score_field = 'score_inv'

    run_df = results_df.sort_values(["query_id", score_field], ascending=[True, False])

    run_df['Q0'] = "Q0"
    run_df['comment'] = retriever_name

    if add_index:
        run_df['index'] = np.tile(list(range(1, how_many_passages_per_question + 1)), results_grouped_df.shape[0])

    output_filename = "{}_run.txt".format(os.path.splitext(results_filename)[0])

    print("output_filename={}".format(output_filename))

    run_df[['query_id', 'Q0', "passage_id", "index", score_field, "comment"]].to_csv(output_filename, header=None, index=False, sep=" ")

    return output_filename, run_df

In [57]:
retrievers_results = []

In [90]:
retrievers_1M_results = []

### Format GPT-4 evaluations to qrels format

In [8]:
qrels_filename = os.path.join(os.path.dirname(ALL_EVALUATIONS), "{}_qrel.txt".format(os.path.splitext(os.path.basename(ALL_EVALUATIONS))[0]))
qrels_1M_filename = os.path.join(os.path.dirname(ALL_EVALUATIONS), "{}_1M_qrel.txt".format(os.path.splitext(os.path.basename(ALL_EVALUATIONS))[0]))

In [89]:
if os.path.exists(qrels_filename) == False:
    eval_df = pd.read_csv(ALL_EVALUATIONS, sep='\t')

    eval_df

    eval_df.drop_duplicates("query_id")[['query_id', 'query']]

    #### Check scores distribution on the current qrels

    np.unique(eval_df['score'].to_numpy(), return_counts=True)

    for group_name, group_df in eval_df.groupby('query_id'):
        print("\nquery_id={}".format(group_name))

        print(np.unique(group_df['score'].to_numpy(), return_counts=True))

    eval_df.groupby('query_id').count()

    eval_df.groupby('query_id').count()['query'].describe()

    #### Prepare the qrels format for trec_eval tools

    eval_df['0'] = '0'

    print("\n\n")
    print(qrels_filename)
    print(qrels_1M_filename)

    eval_df.sort_values('query_id')[['query_id', '0', 'passage_id', 'score']].to_csv(qrels_filename, header=None, index=False, sep=" ")


    #### Select 1M dataset evaluations

    eval_df[eval_df[DATASET_1M_FIELDS].sum(axis=1) > 0].sort_values('query_id')[['query_id', '0', 'passage_id', 'score']].to_csv(qrels_1M_filename, header=None, index=False, sep=" ")

else:
    print("QRELS files ({}) already created...".format(qrels_filename))


query_id=1
(array([0, 1, 2, 3]), array([15, 41, 16, 10]))

query_id=2
(array([0, 1, 2, 3]), array([88,  2,  1,  2]))

query_id=9
(array([0, 1, 2, 3]), array([52, 10, 12,  5]))

query_id=11
(array([0, 1, 2, 3]), array([44, 15,  8,  2]))

query_id=13
(array([0, 1, 2, 3]), array([31, 14, 22,  4]))

query_id=15
(array([0, 1, 2, 3]), array([23, 19,  3, 17]))

query_id=17
(array([0, 1, 2, 3]), array([21,  8,  2, 36]))

query_id=20
(array([0, 1, 2, 3]), array([16, 50, 25,  7]))

query_id=21
(array([0, 1, 2, 3]), array([45, 20,  7, 19]))

query_id=22
(array([0, 1, 2, 3]), array([52, 12,  2,  6]))

query_id=24
(array([0, 1, 2, 3]), array([53, 29, 11,  2]))

query_id=26
(array([0, 1, 2, 3]), array([13,  9, 14, 33]))

query_id=28
(array([0, 1, 2, 3]), array([51, 15, 13,  4]))

query_id=47
(array([0, 1, 2, 3]), array([47, 27, 27,  1]))

query_id=49
(array([0, 1, 2, 3]), array([24, 26, 22,  1]))

query_id=51
(array([0, 1, 2, 3]), array([31, 21, 12,  1]))

query_id=54
(array([0, 1, 2, 3]), array([ 

## 10M dataset

### Compute BM25 nDCG@10

In [58]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} tests/200-queries-passagens-by-BM25-pt_v2_with_query_id_run.txt

In [59]:
retrievers_results.append({'retriever':'BM25', 'nDCG@10': result[0].split('\t')[-1]})

In [60]:
retrievers_results

[{'retriever': 'BM25', 'nDCG@10': '0.4511'}]

### Compute BM25 + E5 nDCG@10

In [61]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {BM25_E5_RUN}

In [62]:
retrievers_results.append({'retriever':'BM25+E5', 'nDCG@10': result[0].split('\t')[-1]})

### Compute BM25 + mT5 nDCG@10

In [63]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {BM25_MT5_RUN}

In [64]:
retrievers_results.append({'retriever':'BM25+mT5', 'nDCG@10': result[0].split('\t')[-1]})

### Compute SPLADE nDCG@10

In [65]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {SPLADE_RUN}

In [66]:
retrievers_results.append({'retriever':'SPLADEv2_pt-BR', 'nDCG@10': result[0].split('\t')[-1]})

### Compute ColBERTX nDCG@10

In [67]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {COLBERTX_RUN}

In [68]:
retrievers_results.append({'retriever':'ColBERT-X_mMARCO_pt-BR', 'nDCG@10': result[0].split('\t')[-1]})

### Compute E5 nDCG@10

In [69]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {E5_RUN}

In [70]:
retrievers_results.append({'retriever':'E5', 'nDCG@10': result[0].split('\t')[-1]})

### Compute E5 base nDCG@10

In [71]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {E5_BASE_RUN}

In [72]:
retrievers_results.append({'retriever':'E5 base', 'nDCG@10': result[0].split('\t')[-1]})

### Compute SPLADE + mT5 nDCG@10

In [73]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {SPLADE_MT5_RUN}

In [74]:
retrievers_results.append({'retriever':'SPLADEv2_pt-BR + mT5', 'nDCG@10': result[0].split('\t')[-1]})

### Compute E5 + ColBERT-X RRF nDCG@10

In [75]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {E5_COLBERTX_RRF_RUN}

In [76]:
retrievers_results.append({'retriever':'E5 + ColBERT-X mMARCO_pt-BR RRF', 'nDCG@10': result[0].split('\t')[-1]})

### Compute E5 + SPLADE RRF nDCG@10

In [77]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {E5_SPLADE_RRF_RUN}

In [78]:
retrievers_results.append({'retriever':'E5 + SPLADEv2_pt-BR RRF', 'nDCG@10': result[0].split('\t')[-1]})

### Compute BM25 1M nDCG@10

In [79]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {BM25_1M_RUN}

retrievers_results.append({'retriever':'BM25 1M', 'nDCG@10': result[0].split('\t')[-1]})

### Compute BM25 1M massiveweb filtered nDCG@10

In [80]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {BM25_1M_MASSIVEWEB_RUN}

retrievers_results.append({'retriever':'BM25 1M massiveweb', 'nDCG@10': result[0].split('\t')[-1]})

### Compute ColBERT-X massiveweb filtered nDCG@10

In [81]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {COLBERTX_1M_MASSIVEWEB_RUN}

retrievers_results.append({'retriever':'ColBERT-X_mMARCO_pt-BR 1M massiveweb', 'nDCG@10': result[0].split('\t')[-1]})

### Prepare ADAv2 massiveweb filtered run and compute nDCG@10

In [54]:
adav2_filename, _ = prepare_run_format(ADAV2, retriever_name="ADAv2")

,query_id,query,passage_id,colbertx_id,position,score,passage
0,1,Qual a maior característica da fauna brasileira?,clueweb22-pt0001-46-05303_17,7028401,1,0.877872,"http://www.klimanaturali.org/2014/03/biomas-e-fitofisionomias-brasileiros.html 8/17 f15/03/2018 BIOMAS E FITOFISIONOMIAS BRASILEIROS: CARACTERÍSTICAS E EVOLUÇÃO DA FAUNA E FLORA - Klima Naturali™ colonização no Brasil e os ciclos de desenvolvimento do país SITE  BRASIL  MUNDO  CIÊNCIAS  GERAL  levaram o homem a ocupar e destruir parte desse espaço. Fauna e Flora Rica em diversidade de espécies, a Mata Atlântica está entre Pesquisa no Site...  as cinco regiões do mundo com maior número de espécies endêmicas. Os animais podem ser considerados generalistas ou especialistas. Os generalistas apresentam hábitos alimentares variados, alta taxa de crescimento e dispersão; vivem em áreas de vegetação aberta e secundária, tolerantes e capazes de aproveitar diferentes recursos oferecidos pelo meio ambiente. Os animais especialistas são mais exigentes em relação aos habitats nos quais vivem, com dieta especí ca, uma alteração no meio ambiente exige dos animais especialistas a procura de novos habitats. Grupos da Mata Atlântica Mamíferos – A Mata Atlântica possui cerca de 250 espécies de mamíferos, das quais 55 são endêmicas. Os mamíferos são os"
1,1,Qual a maior característica da fauna brasileira?,clueweb22-pt0000-17-01771_1,827050,2,0.875344,"O Brasil é um país conhecido mundialmente por suas belezas naturais, incluindo a sua fauna que contém diversas espécies de animais exóticos como a arara-azul, o mico-leão-dourado e a onça-pintada. Há ... Redação completa Enviada em: 25/04/2017 A vasta natureza brasileira, exaltada desde o século dezesseis na carta de Pero Vaz de Caminha, continua sendo um dos maiores patrimônios nacionais, mas é também o maior alvo do contrabando e das dest... Redação completa Enviada em: 26/04/2017 Em vários locais do mundo, principalmente no Brasil, agrega inúmeras espécies de animais em ecossistemas e climas distintos. A essência atualizante da sociedade traz como característica admirável, a c... Redação completa Enviada em: 26/04/2017 Nos primórdios do século XVI,os lusitanos desembarcaram nas terras brasileiras já com o ideal de exploração.Nesse contexto,até mesmo os animais silvestres eram incorporados nas práticas mercantilistas... Redação completa Enviada em: 26/04/2017 Para Paul Atson, co-fundador do Greenspace, \""a inteligência é a habilidade de se viver em harmonia com o meio ambiente\"". Desse modo, percebe-se que essa harmonia é rompida no Brasil com o tráfico de a... Redação completa Enviada em: 27/04/2017 Tráfico de animais silvestres: um problema secular"
2,1,Qual a maior característica da fauna brasileira?,clueweb22-pt0000-17-01771_19,827062,3,0.875237,"A fauna brasileira está entre as mais diversas do mundo. Entretanto, toda essa riqueza vem sendo constantemente ameaçada devido a execrável prática de comércio ilícito de diversas espécies. Desse modo... Redação completa Enviada em: 02/09/2017 Vida Os animais da fauna brasileira são variados, diversificados em cores, formas e espécies, possuem uma pluralidade única, em todo o mundo, e por conseguinte, atraem a atenção daqueles,... Redação completa Enviada em: 07/09/2017 Desde o início do século XVI, com a chegada oficial da colonização portuguesa ao Brasil, cultiva-se a ideia de que a fauna e flora da pátria são infinitas. Nesse sentido o homem começou a apropriar-se... Redação completa Enviada em: 08/09/2017 Tráfico. Roubo. Desenvolvimento. Progresso, são pilares da biopirrataria. Hodiernamente, o número de espécies silvestres contrabandeadas tem uma tendência ao aumento, pois os países desenvolvidos gara... Redação completa Enviada em: 14/09/2017 Relativo o combate ao comércio ilegal de animais silvestres, é possível afirmar que muitos países como o Brasil, apresentam uma rica biodiversidade, porém com o tráfico de animais silvestres ocorrendo... Redação completa Enviada em: 12/09

output_filename=tests/adav2_1M_docs_000_999_sampled_50_queries_run.txt


In [82]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {adav2_filename}

retrievers_results.append({'retriever':'ADA v2 1M massiveweb', 'nDCG@10': result[0].split('\t')[-1]})

### Check the overall results

In [83]:
pd.DataFrame(retrievers_results)

,retriever,nDCG@10
0,BM25,0.4511
1,BM25+E5,0.6427
2,BM25+mT5,0.7180
3,SPLADEv2_pt-BR,0.5852
4,ColBERT-X_mMARCO_pt-BR,0.6336
5,E5,0.5619
6,E5 base,0.4009
7,SPLADEv2_pt-BR + mT5,0.7035
8,E5 + ColBERT-X mMARCO_pt-BR RRF,0.6445
9,E5 + SPLADEv2_pt-BR RRF,0.6344


## 1M dataset

### Compute BM25 1M massiveweb filtered nDCG@10

In [91]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_1M_filename} {BM25_1M_MASSIVEWEB_RUN}

retrievers_1M_results.append({'retriever':'BM25 1M massiveweb', 'nDCG@10': result[0].split('\t')[-1]})

### Compute ColBERT-X massiveweb filtered nDCG@10

In [92]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_1M_filename} {COLBERTX_1M_MASSIVEWEB_RUN}

retrievers_1M_results.append({'retriever':'ColBERT-X_mMARCO_pt-BR 1M massiveweb', 'nDCG@10': result[0].split('\t')[-1]})

### Compute ADA v2 massiveweb filtered nDCG@10

In [93]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_1M_filename} {adav2_filename}

retrievers_1M_results.append({'retriever':'ADA v2 1M massiveweb', 'nDCG@10': result[0].split('\t')[-1]})

### Check the overall results

In [94]:
pd.DataFrame(retrievers_1M_results)

,retriever,nDCG@10
0,BM25 1M massiveweb,0.4445
1,ColBERT-X_mMARCO_pt-BR 1M massiveweb,0.5347
2,ADA v2 1M massiveweb,0.6286
